In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import callbacks

In [2]:
# 데이터 로딩
mpgDF = pd.read_csv('../data/auto-mpg.csv')
mpgDF.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model-year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


In [3]:
# 결측치 제거
mpgDF = mpgDF.dropna()

In [4]:
# 훈련 데이터 준비
target = mpgDF['mpg']
data = mpgDF.drop(['mpg'], axis=1)
data.head()

,cylinders,displacement,horsepower,weight,acceleration,model-year
0,8,307.0,130.0,3504,12.0,70
1,8,350.0,165.0,3693,11.5,70
2,8,318.0,150.0,3436,11.0,70
3,8,304.0,150.0,3433,12.0,70
4,8,302.0,140.0,3449,10.5,70


In [5]:
# 모델 설계
mpgModel = Sequential(name = 'mpg_model')

In [6]:
# 입력층
mpgModel.add(Dense(32, input_shape=(data.columns.nunique(),), activation='relu', name = 'In'))

# 은닉층
mpgModel.add(Dense(16, activation='relu', name = 'Hidden1'))
mpgModel.add(Dense(8, activation='relu', name = 'Hidden2'))

# 출력층
mpgModel.add(Dense(1, name = 'Out'))

In [7]:
mpgModel.summary()

Model: "mpg_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 In (Dense)                  (None, 32)                224       
                                                                 
 Hidden1 (Dense)             (None, 16)                528       
                                                                 
 Hidden2 (Dense)             (None, 8)                 136       
                                                                 
 Out (Dense)                 (None, 1)                 9         
                                                                 
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 모델 컴파일
mpgModel.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mae','mse'])

In [9]:
# 모델 학습
EARLY_STOPPING = callbacks.EarlyStopping(monitor = 'val_loss',patience=10)
history = mpgModel.fit(data, target, 
                       epochs=500, 
                       validation_split=0.2, 
                       verbose=1
                       ,callbacks=[EARLY_STOPPING])

Epoch 1/500
10/10 [==============================] - 1s 35ms/step - loss: 4089.8101 - mae: 53.2696 - mse: 4089.8101 - val_loss: 279.3044 - val_mae: 14.3325 - val_mse: 279.3044
Epoch 2/500
10/10 [==============================] - 0s 5ms/step - loss: 940.8528 - mae: 25.5274 - mse: 940.8528 - val_loss: 171.0131 - val_mae: 9.6320 - val_mse: 171.0131
Epoch 3/500
10/10 [==============================] - 0s 5ms/step - loss: 528.3829 - mae: 17.3909 - mse: 528.3829 - val_loss: 145.6901 - val_mae: 9.0138 - val_mse: 145.6901
Epoch 4/500
10/10 [==============================] - 0s 5ms/step - loss: 335.6634 - mae: 14.6577 - mse: 335.6634 - val_loss: 132.0132 - val_mae: 9.3286 - val_mse: 132.0132
Epoch 5/500
10/10 [==============================] - 0s 5ms/step - loss: 242.6366 - mae: 11.3520 - mse: 242.6366 - val_loss: 135.2530 - val_mae: 8.6653 - val_mse: 135.2530
Epoch 6/500
10/10 [==============================] - 0s 5ms/step - loss: 132.0519 - mae: 8.7343 - mse: 132.0519 - val_loss: 143.8650 - v

In [10]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

In [11]:
def plot_history():
    
    plt.figure(figsize=(10, 12))
    
    plt.subplot(2, 1, 1)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error  [MAE]')
    plt.plot(hist['epoch'], hist['mae'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'], label='Val Error')
    plt.ylim([0,5])
    plt.title("[MAE]")
    plt.legend()
    
    
    plt.subplot(2, 1, 2)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MSE^2$]')
    plt.plot(hist['epoch'], hist['mae'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'], label='Val Error')
    plt.ylim([0,5])
    plt.title("[$MSE^2$]")
    plt.legend()
    plt.show()

In [12]:
loss, mae, mse = mpgModel.evaluate(data, target, verbose=1)

print(f"\n테스트 세트의 평균 절대 오차: {mae:5.2f}")

13/13 [==============================] - 0s 1ms/step - loss: 54.9688 - mae: 5.7716 - mse: 54.9688

테스트 세트의 평균 절대 오차:  5.77


In [ ]:
# 모델 저장
mpgModel.save('mpg_model.h5')
# mpgModel.save(mpg_model.keras)

# 모델 가중치 저장
# mpgModel.save_weights('mpg_model_weights.h5')